
... ***CURRENTLY UNDER DEVELOPMENT*** ...


### TODO
- CALCULAR LEVEL NEARSHORE PARA HIST Y SIM
- GESTIONAR LOS 15 "CASOS" HYCREW PARA EL OUTPUT
- GESTIONAR LOS 15 CASOS HYCREW X 10 SIMULACIONES TESLAKIT PARA EL OUTPUT
- REFACTOR FUNCIONES REPETIDAS HYCREW

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Interpolation
from teslakit.mda import Normalize



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


# reef characteristics
reef_cs = {
    'rslope': 0.0505,
    'bslope': 0.1667,
    'rwidth': 250,
    'cf': 0.0105,
}


# load Hycreww RBF coefficients and sim. variables min. and max.
var_lims, rbf_coeffs = db.Load_HYCREWW()


In [3]:
# hycreww interpolation 
def hycreww_runup(var_lims, rbf_coeffs, dset):
    '''
    Calculates RunUp using hycreww RBFs
    
    var_lims   - hycreww variables min and max limits
    rbf_coeffs - hycreww rbf coefficients
    dset       - input dataset (pandas.DataFrame with "rbf_vns" columns)
    
    '''
    
    # RBF wave conditions 
    rbf_hs = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
    rbf_hs_lo = [0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05 ]
    rbf_vns = ['level', 'rslope', 'bslope', 'rwidth', 'cf']
    
    # RBF parameters
    ix_sc = [0, 1, 2, 3, 4]
    ix_dr = []
    minis = [var_lims[x][0] for x in rbf_vns]
    maxis = [var_lims[x][1] for x in rbf_vns]
    
    # RBF dataset to interpolate
    ds_in = dset[rbf_vns]

    # normalize data
    ds_nm ,_ ,_ = Normalize(ds_in.values, ix_sc, ix_dr, minis=minis, maxis=maxis)

    # RBF interpolation (with all cases?)
    ru_out = []
    for rc in rbf_coeffs:
        ro = RBF_Interpolation(rc['constant'], rc['coeff'], rc['nodes'], ds_nm.T)
        ru_out.append(ro)
    ru_z = np.array(ru_out)
    
    # RU Linear interpolation (hs, hs_lo -> runup)
    RU = []
    for c, vn in enumerate(rbf_vns):
        vq = griddata((rbf_hs, rbf_hs_lo), ru_z[:,c], (dset['hs'], dset['hs_lo2']), method='linear')
        RU.append(vq)
        
    return RU



## Hycreww RBF Interpolation: Historical

In [4]:
# Load complete historical data
data = db.Load_HIST_Complete_daily()

# prepare dataset for hycreww: waves + reef data
dset = data[['Hs', 'Tp', 'Dir']].to_dataframe()
dset = dset.rename(columns={"Hs": "hs", "Tp": "tp", 'Dir':'dir'})
dset['hs_lo2'] = dset['hs']/(1.5613*dset['tp']**2)

# TODO: en historical no tengo level, no tengo ss, no tengo TWL ni AWL... (?)
dset['level']= 0.5  # TODO usar level correcto

# add reef characteristics
for p in reef_cs.keys():
    dset[p] = reef_cs[p]

# discard data outside limits
for vn in var_lims.keys():
    dset = dset[(dset[vn] > var_lims[vn][0]) &(dset[vn] < var_lims[vn][1])]
    
# calculate runup with hycreww
RU = hycreww_runup(var_lims, rbf_coeffs, dset)

# store runup 
dset_out = dset.copy()
dset_out['runup'] = dset_out['level'] + RU[0]

print(dset_out)

# TODO: store 15 RU output with n_rbf dim ?


                 hs        tp        dir    hs_lo2  level  rslope  bslope  \
time                                                                        
1996-01-16  2.54812  13.85190   6.048340  0.008506    0.5  0.0505  0.1667   
1996-01-17  3.29111  15.96870   6.711730  0.008266    0.5  0.0505  0.1667   
1996-01-18  2.91396  14.51940  32.691399  0.008853    0.5  0.0505  0.1667   
1996-01-19  2.26963   8.61673  46.048000  0.019579    0.5  0.0505  0.1667   
1996-01-20  2.29604   8.79586  52.621799  0.019008    0.5  0.0505  0.1667   
...             ...       ...        ...       ...    ...     ...     ...   
2014-01-12  2.10927   9.69291  53.679699  0.014379    0.5  0.0505  0.1667   
2014-01-13  2.09039   9.30574  61.769199  0.015461    0.5  0.0505  0.1667   
2014-01-14  2.00630   8.58236  60.814800  0.017446    0.5  0.0505  0.1667   
2014-01-15  2.53932  10.20110  45.628799  0.015629    0.5  0.0505  0.1667   
2014-01-16  2.58802   8.19699  50.468300  0.024670    0.5  0.0505  0.1667   


## Hycreww RBF Interpolation: Simulation

In [5]:
# Load complete simulation data
data = db.Load_SIM_Complete_daily()
data = data.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})
data['hs_lo2'] = data['hs']/(1.5613*data['tp']**2)


# solve all teslakit simulations
for s in data.n_sim:
    ds = data.sel(n_sim=s).to_dataframe()  # select simulation

    # TODO usar level correcto
    ds['level']= 0.5  

    # add reef characteristics
    for p in reef_cs.keys():
        ds[p] = reef_cs[p]

    # discard data outside limits
    for vn in var_lims.keys():
        ds = ds[(ds[vn] > var_lims[vn][0]) &(ds[vn] < var_lims[vn][1])]
        
    # calculate runup with hycreww
    RU = hycreww_runup(var_lims, rbf_coeffs, ds) 

    # store runup 
    ds_out = ds.copy()
    ds_out['runup'] = ds_out['level'] + RU[0]  #! 

    print(dset_out)
    
    
    # TODO: store 15 RU output with n_rbf dim... for each teslakit simulation?
    import sys; sys.exit()
    

                 hs        tp        dir    hs_lo2  level  rslope  bslope  \
time                                                                        
1996-01-16  2.54812  13.85190   6.048340  0.008506    0.5  0.0505  0.1667   
1996-01-17  3.29111  15.96870   6.711730  0.008266    0.5  0.0505  0.1667   
1996-01-18  2.91396  14.51940  32.691399  0.008853    0.5  0.0505  0.1667   
1996-01-19  2.26963   8.61673  46.048000  0.019579    0.5  0.0505  0.1667   
1996-01-20  2.29604   8.79586  52.621799  0.019008    0.5  0.0505  0.1667   
...             ...       ...        ...       ...    ...     ...     ...   
2014-01-12  2.10927   9.69291  53.679699  0.014379    0.5  0.0505  0.1667   
2014-01-13  2.09039   9.30574  61.769199  0.015461    0.5  0.0505  0.1667   
2014-01-14  2.00630   8.58236  60.814800  0.017446    0.5  0.0505  0.1667   
2014-01-15  2.53932  10.20110  45.628799  0.015629    0.5  0.0505  0.1667   
2014-01-16  2.58802   8.19699  50.468300  0.024670    0.5  0.0505  0.1667   

SystemExit: 